In [1]:
import psycopg2
import params

In [2]:
try:
    conn = psycopg2.connect(
        host = params.host, 
        dbname = params.dbname, 
        user = params.user, 
        password = params.password, 
        port = params.port)
    cur = conn.cursor()

except (Exception, psycopg2.Error) as error:
    print("Error en la apertura de base de datos: \n\t{}".format(error))
    conn = psycopg2.connect(
        host = params.host, 
        dbname = 'postgres', 
        user = params.user, 
        password = params.password, 
        port = params.port)
    cur = conn.cursor()
    
    conn.set_session(autocommit=True)
    cur.execute("CREATE DATABASE repartos_test;")

    conn = psycopg2.connect(
        host = params.host, 
        dbname = params.dbname, 
        user = params.user, 
        password = params.password, 
        port = params.port)
    cur = conn.cursor()

In [7]:
try:
    cur.execute("""CREATE TABLE IF NOT EXISTS clientes(
        id SERIAL PRIMARY KEY,
        nombre VARCHAR(50) NOT NULL,
        email VARCHAR(50) NOT NULL,
        direccion VARCHAR(250) NOT NULL,
        latitud NUMERIC(9,6) NOT NULL,
        longitud NUMERIC(9,6) NOT NULL
    );
    """)
    conn.commit()
except Exception:
    conn.rollback()
    raise Exception

In [8]:
try:
    cur.execute("""CREATE TABLE IF NOT EXISTS comercios(
        id SMALLSERIAL PRIMARY KEY,
        nombre VARCHAR(50) NOT NULL,
        zona SMALLINT NOT NULL,
        direccion VARCHAR(250) NOT NULL,
        tipo VARCHAR(50) NOT NULL,
        latitud NUMERIC(9,6) NOT NULL,
        longitud NUMERIC(9,6) NOT NULL
    );
    """)
    conn.commit()
except Exception:
    conn.rollback()
    raise Exception

In [9]:
try:
    cur.execute("""ALTER TABLE comercios 
        ADD COLUMN IF NOT EXISTS 
            horario_L VARCHAR(50),
        ADD COLUMN IF NOT EXISTS 
            horario_M VARCHAR(50),
        ADD COLUMN IF NOT EXISTS 
            horario_X VARCHAR(50),
        ADD COLUMN IF NOT EXISTS 
            horario_J VARCHAR(50),
        ADD COLUMN IF NOT EXISTS 
            horario_V VARCHAR(50),
        ADD COLUMN IF NOT EXISTS 
            horario_S VARCHAR(50),
        ADD COLUMN IF NOT EXISTS 
            horario_D VARCHAR(50);
    """)
    conn.commit()
except Exception:
    conn.rollback()
    raise Exception

In [10]:
try:
    cur.execute("""CREATE TABLE IF NOT EXISTS repartidores(
        id SMALLSERIAL PRIMARY KEY NOT NULL,
        nombre VARCHAR(50) NOT NULL,
        email VARCHAR(50) NOT NULL,
        status BOOLEAN NOT NULL,
        vehiculo VARCHAR(50) NOT NULL,
        ocupado BOOLEAN,
        zona SMALLINT,
        latitud NUMERIC(9,6),
        longitud NUMERIC(9,6),
        datetime_ult_act TIMESTAMP
        );
        """)

    conn.commit()
except Exception:
    conn.rollback()
    raise Exception

In [11]:
try:
    cur.execute("""CREATE TABLE IF NOT EXISTS pedidos(
        id_pedido SERIAL PRIMARY KEY NOT NULL,
        id_cliente INTEGER NOT NULL,
        id_comercio SMALLINT NOT NULL,
        id_repartidor SMALLINT,
        latitud NUMERIC(9,6) NOT NULL,
        longitud NUMERIC(9,6) NOT NULL,
        datetime_pedido TIMESTAMP NOT NULL,
        tamaño REAL,
        FOREIGN KEY (id_cliente) REFERENCES clientes(id),
        FOREIGN KEY (id_comercio) REFERENCES comercios(id),
        FOREIGN KEY (id_repartidor) REFERENCES repartidores(id)
        );
        """)
    conn.commit()
except Exception:
    conn.rollback()
    raise Exception

In [3]:
cur.close()
conn.close()